In [255]:
#Import Python Libraries
import numpy as np
import pandas as pd
import tweepy
import csv
import re
import string 

In [256]:
#Read csv file
df = pd.read_csv('newExtractedData (1) (1) (1).csv')

In [257]:
df.dtypes

date         object 
username     object 
to           object 
replies      int64  
retweets     int64  
favorites    int64  
text         object 
geo          float64
mentions     object 
hashtags     object 
id           float64
permalink    object 
stance       object 
dtype: object

In [258]:
df.head(10)

,date,username,to,replies,retweets,favorites,text,geo,mentions,hashtags,id,permalink,stance
0,07/03/2020,XHrb9,AlRajhiBankCare,1,0,0,السلام عليكم هل اقدر ابيع جزء من اسهم اكتتاب ارامكو ؟ من تطبيق المباشر للافراد,NaN,NaN,NaN,1.236390e+18,https://twitter.com/XHrb9/status/1236391193879220225,Neutral
1,07/03/2020,metrekmeshal,NaN,17,3,6,#تاسي مساء الخير ... غداً تستأنف التداولات في السوق السعودي بعد فشل اتفاق اوبك بلس وهبوط اسعار النفط لمستويات متدنية جديدة * توقعاتكم للاسبوع المقبل ؟ كتوقع شخصي ١. كسر سعر اكتتاب ارامكو . ٢. كسر مستوى ٧٠٠٠ نقطة . ٣. هناك أسهم ستعاكس اتجاه السوق موفقين,NaN,NaN,NaN,1.236240e+18,https://twitter.com/metrekmeshal/status/1236236655792656384,Neutral
2,07/03/2020,Nawaf44930655,vip9tasi,0,0,0,والله يابو دانة لعنة انهيار السوق ٢٠٠٦ حتى اليوم ملاحقة سوقنا ، ومع اكتتاب ارامكو عدد كبير من الناس اكتتبو وصارو يتابعون السوق وأصبح في اهتمام في الفرص الي فيه وللأسف خابت توقعات الكثير منهم ، اعتقد نحتاج فتره زمنية اطول وتحسن الأوضاع الاقتصادية علشان نصل للمستويات الي ذكرها الاخ,NaN,NaN,NaN,1.236230e+18,https://twitter.com/Nawaf44930655/status/1236233044765347841,Negative
3,07/03/2020,A5O_FAZZAH,DrAbdullahGh_,0,0,0,استثمار مادي في العلاقه ! هذا اكتتاب ارامكو مو حب,NaN,NaN,NaN,1.236230e+18,https://twitter.com/A5O_FAZZAH/status/1236227794004324352,Neutral
4,07/03/2020,hamimi50,alwardit,0,0,1,بالنسبة للميزانية السعوديه فقد خرجت من أزمة اسعار النفط على الأقل للخمس سنوات القادمه بسبب اكتتاب أرامكو و الخطه الاستثمارية الشجاعة التي اعتمدتها في السبع سنوات الاخيره وعقبال عندنا,NaN,NaN,NaN,1.236200e+18,https://twitter.com/hamimi50/status/1236201906030153733,Positive
5,07/03/2020,ArkadCo,seeeen881,1,0,0,١٠٠٠٪ نصب و احتيال .. نفس الطريقة كانوا يعملوها قلبل اكتتاب ارامكو ..,NaN,NaN,NaN,1.236180e+18,https://twitter.com/ArkadCo/status/1236175759812812800,Neutral
6,06/03/2020,Icho75,BarjiLarbi,1,0,0,ماهي العلاقة بين ارامكو والبريكسيت؟ ولماذا اكتتاب ارامكوا مربوط باستنتاج الانهيار ؟ لو تكرمت وضح الأمر يحتاج لتفصيل...,NaN,NaN,NaN,1.236010e+18,https://twitter.com/Icho75/status/1236005463457755141,Neutral
7,06/03/2020,viiliivx,NaN,0,0,0,طبعاً هذا كان قبل اكتتاب أرامكو بعدها وصلت السعودية ل المركز التاسع على مستوى العالم و السابعه دولياً و تحققت رؤية ٢٠٣٠ ب نهاية ٢٠١٩,NaN,NaN,NaN,1.235950e+18,https://twitter.com/viiliivx/status/1235952334267650048,positive
8,06/03/2020,nabaatv,NaN,0,1,1,#أرامكو تخطط لطرح اكتتاب جديد خارج #الرياض.. فما الخطورة في ذلك تقرير: محسن العلوي http://shorturl.at/dipA4 #السعودية,NaN,NaN,NaN,1.235890e+18,https://twitter.com/nabaatv/status/1235891020740669440,Neutral
9,06/03/2020,wolf0e,NaN,0,0,0,وانا كل مره ضحيه لهذي المخـnـثه اخر مره يدق علي تريليون رقم اخر شي طلع حق اكتتاب ارامكو مدري ايش سخافتهم يروجون على حساب ازعاج الخلق والبثاره عيال الكلب بصير اغنيه وادعس شركتكم,NaN,NaN,NaN,1.235860e+18,https://twitter.com/wolf0e/status/1235855694848376832,Neutral


In [259]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [260]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

In [261]:
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [269]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [270]:
#start processing the tweet
def processPost(tweet):

    #Replace @username with empty string
    tweet = re.sub('@[^\s]+', ' ', str(tweet))
    
    #Convert www.* or https?://* to " "
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',str(tweet))
    
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)

    # remove punctuations
    tweet= remove_punctuations(str(tweet))
    
    # normalize the tweet
    tweet= normalize_arabic(str(tweet))
    
    # remove repeated letters
    tweet=remove_repeating_char(tweet)

    #Remove new lines with white space
    tweet = str(tweet).replace('\n',' ')
    tweet = str(tweet).replace('\t',' ')
    
    #remove emoji
    tweet = remove_emoji(tweet)
        
    return tweet

In [271]:
# remove Arabic stop words from the tweets
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
ar_stops = set(stopwords.words('arabic'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [272]:
# remove Arabic stop words from the tweets
for word in df['text']:
    if word in ar_stops:        
        words_filtered.remove(word)

In [273]:
df["clean text"] = df['text'].apply(lambda x: processPost(x))

In [274]:
df.tail(5)

,date,username,to,replies,retweets,favorites,text,geo,mentions,hashtags,id,permalink,stance,clean text
5723,05/12/2019,AlamlRoooo7,NaN,0,0,1,#اكتتاب_أرامكو هو الأكبر بالعالم • إجمالي الأموال المجمعة 446 مليار • تخصيص 1500 سهم للمكتتب الفرد الواحد في #ارامكوا,NaN,NaN,NaN,1.202710e+18,https://twitter.com/AlamlRoooo7/status/1202707852235870208,positive,اكتابارامكو هو الاكبر بالعالم • اجمالي الاموال المجمعه 46 مليار • تخصيص 150 سهم لمكتب الفرد الواحد في ارامكوا
5724,05/12/2019,KBri32457754,NaN,0,0,1,#اكتتاب_أرامكو يصبح الأكبر في العالم بقيمة 25.6 مليار دولار • تخطّت قيمة #طلبات_الاكتتاب 119 مليار دولار,NaN,NaN,NaN,1.202710e+18,https://twitter.com/KBri32457754/status/1202707814860414977,positive,اكتابارامكو يصبح الاكبر في العالم بقيمه 256 مليار دولار • تخطّت قيمه طلباتالاكتاب 19 مليار دولار
5725,05/12/2019,0123Bin,NaN,1,1,1,ارتفاع عدد المكتتبين الأفراد في #أرامكو لأكثر من 5 ملايين. (الإخبارية) #اكتتاب_ارامكو,NaN,NaN,NaN,1.202710e+18,https://twitter.com/0123Bin/status/1202707751996137472,Neutral,ارتفاع عد المكتبين الافراد في ارامكو لاكثر من 5 ملاين الاخباريه اكتابارامكو
5726,05/12/2019,Esam7736,NaN,0,0,0,#اكتتاب_ارامكو بما اني احد ملاك اسهم في شركة ارامكوا اذا عبيتو سفينه امريكا من شركتنا وصوهم المره الثانيه يجيبون معهم كروز ملبورو اصلي واخصموه من الارباح,NaN,NaN,NaN,1.202710e+18,https://twitter.com/Esam7736/status/1202707750951800837,Neutral,اكتابارامكو بما اني احد ملاك اسهم في شركه ارامكوا اذا عبيتو سفينه امريكا من شركتنا وصوهم المره الثانيه يجيبون معهم كروز ملبورو اصلي واخصموه من الارباح
5727,05/12/2019,alqabas,NaN,1,4,2,NaN,NaN,NaN,NaN,1.202710e+18,https://twitter.com/alqabas/status/1202707716348809216,positive,nan


In [275]:
pd.set_option('display.max_colwidth', -1)  
print(df.iloc[528])

date          06/01/2020                                                                  
username      alfataalmuasser                                                             
to            dbasdosari1                                                                 
replies       0                                                                           
retweets      0                                                                           
favorites     0                                                                           
text          عساك تفوزين بس انتي عاد احسك ماتلعبين الا برهان اكتتاب ارامكو شكله جاء منها☻
geo           NaN                                                                         
mentions      NaN                                                                         
hashtags      NaN                                                                         
id            1.21405e+18                                                                 